In [1]:
#necessary imports
import pandas as pd
import numpy as np

from datetime import datetime

In [2]:
#create a dataframe to hold the final parsed values
pd.set_option('display.max_rows', None)
colnames = [
    'user', 'date', 'numPCwithUSBDay', 'numPCwithUSBNight', 'numConnectionDay', 
    'numConnectionNight', 'numCopy2DeviceDay', 'numCopy2DeviceNight', 'numWrite2DeviceDay',
    'numWrite2DeviceNight', 'numCopyFromDeviceDay', 'numCopyFromDeviceNight', 'numWriteFromDeviceDay',
    'numWriteFromDeviceNight', 'numDelFromDeviceDay', 'numDelFromDeviceNight', 'numOpenOnPCDay',
    'numOpenOnPCNight'
]

results = pd.DataFrame(columns = colnames)

In [3]:
df = pd.read_csv('device.csv', usecols = ['user'])
device_users = df.user.unique().tolist()

In [4]:
#file holding separated date file names
file = open('file_data/file_files.log', 'r')

#loop through date files
while True:
    line = file.readline().strip()
    
    #if there is no line, leave the loop
    if not line:
        break
    
    #read the needed files
    df = pd.read_csv(f'file_data/{line}')
    current_date = df.only_date.iloc[0]
    device_data = pd.read_csv(f'device_data/device-{current_date}.csv')
    
    #group the data by user
    grouped = df.groupby('user')
    
    for user, group in grouped:
    
        numConnectionDay = 0
        numConnectionNight = 0

        if user in device_users:

            numConnectionDay = len(device_data.loc[(device_data.WorkingHours == 'Work Hours') & (device_data.activity == "Connect") & (device_data.user == user)])
            numConnectionNight = len(device_data.loc[(device_data.WorkingHours == 'Off Hours') & (device_data.activity == "Connect") & (device_data.user == user)])


        attributes = {
            'user' : user,
            'date' : current_date,
            'numPCwithUSBDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & ((group.to_removable_media == True) | (group.from_removable_media == True))]),
            'numPCwithUSBNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & ((group.to_removable_media == True) | (group.from_removable_media == True))]),
            'numConnectionDay' : numConnectionDay,
            'numConnectionNight' : numConnectionNight,
            'numCopy2DeviceDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'File Copy') & (group.to_removable_media == True)]),
            'numCopy2DeviceNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'File Copy') & (group.to_removable_media == True)]),
            'numWrite2DeviceDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'File Write') & (group.to_removable_media == True)]),
            'numWrite2DeviceNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'File Write') & (group.to_removable_media == True)]),
            'numCopyFromDeviceDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'File Copy') & (group.from_removable_media == True)]),
            'numCopyFromDeviceNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'File Copy') & (group.from_removable_media == True)]),
            'numWriteFromDeviceDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'File Write') & (group.from_removable_media == True)]),
            'numWriteFromDeviceNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'File Write') & (group.from_removable_media == True)]),
            'numDelFromDeviceDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'File Delete') & (group.from_removable_media == True)]),
            'numDelFromDeviceNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'File Delete') & (group.from_removable_media == True)]),
            'numOpenOnPCDay' : len(group.loc[(group.WorkingHours == 'Work Hours') & (group.activity == 'File Open')]),
            'numOpenOnPCNight' : len(group.loc[(group.WorkingHours == 'Off Hours') & (group.activity == 'File Open')]),
        }

        results.loc[len(results)] = [attributes[column] for column in colnames]

In [12]:
results.to_csv('new_parsed_device.csv')